In [1]:
from dotenv import load_dotenv
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain.vectorstores import FAISS

import os
import pickle

In [2]:
# Load index from file

In [3]:
load_dotenv()

True

In [4]:
loaded_faiss_vs = FAISS.load_local(
    folder_path="./vectordb/faiss/nfcorpus/",
    embeddings=OpenAIEmbeddings())

In [5]:
retriever = loaded_faiss_vs.as_retriever(search_kwargs={'k': 5})

In [6]:
# Define the RAG pipeline

In [7]:
model = 'gpt-3.5-turbo-instruct'
llm = OpenAI(model_name=model, openai_api_key=os.getenv("OPENAI_API_KEY"))

In [8]:
template = """Answer the question or Explain the topic given this additional context: {context}
Question: {question}"""

In [9]:
prompt = ChatPromptTemplate.from_template(template)

In [10]:
docs_file_path = './dataset/nfcorpus/documents.pkl'
with open(docs_file_path, 'rb') as file:
    docs = pickle.load(file)
print("Documents loaded successfully.")

Documents loaded successfully.


In [11]:
def format_docs(_docs):
    ls = []
    for doc in _docs:
        if doc.page_content in docs:
            ls.append(docs[doc.page_content]["text"][:800])
    return ls

In [12]:
chain = ({"context": retriever | format_docs, "question": RunnablePassthrough()} 
         | prompt 
         | llm 
         | StrOutputParser())

In [13]:
# Run the RAG pipeline

In [14]:
query_file_path = './openai_embeddings/nfcorpus/query_embeddings.pkl'
with open(query_file_path, 'rb') as file:
    loaded_queries = pickle.load(file)
print("Query embeddings loaded successfully.")

Query embeddings loaded successfully.


In [ ]:
loaded_queries["PLAIN-2"]["text"]

In [15]:
chain.invoke(loaded_queries["PLAIN-2"]["text"])

'\n\nAnswer: The relationship between statin use and breast cancer risk is still inconclusive, with conflicting evidence from various epidemiological studies. While some studies suggest that statins may decrease the risk of breast cancer, others have found no significant association or even potential harmful effects. To better understand this relationship, a meta-analysis was conducted which combined data from multiple observational studies. Subgroup analyses, sensitivity analysis, and cumulative analyses were also performed, but no clear conclusion could be drawn. Additionally, a population-based case-control study and a large cohort study both found no significant association between statin use and breast cancer risk. Therefore, the current evidence does not support the claim that statin drugs cause breast cancer.'

In [38]:
loaded_queries["PLAIN-23"]["text"]

'How to Reduce Exposure to Alkylphenols Through Your Diet'

In [39]:
chain.invoke(loaded_queries["PLAIN-23"]["text"])

'\n\nOne potential way to reduce exposure to alkylphenols through your diet is by implementing a trial period of eliminating artificial colorings and preservatives. This is supported by evidence that suggests polyphenols, found in plants and foods such as cabbage, broccoli, green tea, chili peppers, apples, red wine, peanuts, and turmeric, have anti-inflammatory properties that can lower the risk of cardiovascular disease and other chronic diseases. Additionally, following a diet rich in polyphenols may also help decrease exposure to other environmental contaminants, such as antibiotics and phthalates, which can be inadvertently consumed through food. There is ongoing research on the potential therapeutic effects of polyphenols in managing allergic disorders, which may further contribute to reducing exposure to alkylphenols through the diet.'

In [40]:
loaded_queries["PLAIN-33"]["text"]

'What’s Driving America’s Obesity Problem?'

In [41]:
chain.invoke(loaded_queries["PLAIN-33"]["text"])

'\n\nThe major drivers of the obesity epidemic are increased energy intake and reduced physical activity, which have led to a reliance on processed, energy-dense foods and a decrease in physical activity levels. This shift in diet and activity began in the 1970s and has been seen in both low and high income countries. This has resulted in a rise in obesity and related diseases such as diabetes, heart disease, and cancer. To address this issue, a global strategy is needed to promote healthy eating and increase physical activity levels. Failure to address this issue may lead to a significant increase in obesity-related health and economic consequences in the next two decades.'

In [42]:
loaded_queries["PLAIN-78"]["text"]

'What Do Meat Purge and Cola Have in Common?'

In [43]:
chain.invoke(loaded_queries["PLAIN-78"]["text"])

'\n\nThey are both associated with an increased risk of colorectal cancer.'

In [44]:
loaded_queries["PLAIN-143"]["text"]

'Are Dental X-Rays Safe?'

In [45]:
chain.invoke(loaded_queries["PLAIN-143"]["text"])

'\n\nAnswer: While diagnostic imaging, including dental x-rays, is an important tool in modern healthcare, there are potential risks associated with exposure to ionizing radiation. The National Council on Radiation Protection & Measurements, Centers for Disease Control and Prevention, and U.S. Food and Drug Administration have all published recommendations and guidelines for reducing these risks in dental settings. These include selecting patients carefully and using low-dose techniques, such as digital sensors and collimation, to minimize exposure. However, a recent study suggests a potential link between dental x-rays and brain tumors, highlighting the importance of informed decision making and proper communication of risks to patients. Overall, while the risk of developing cancer from diagnostic radiation is low, it is important for dentists to follow recommended protocols and for patients to be fully informed about the potential risks.'

In [46]:
loaded_queries["PLAIN-165"]["text"]

'Breast Cancer & Alcohol: How Much is Safe?'

In [47]:
chain.invoke(loaded_queries["PLAIN-165"]["text"])

'\n\nAnswer: There is conflicting evidence on the relationship between alcohol consumption and breast cancer risk. Some studies have shown that moderate to heavy alcohol consumption may increase the risk of breast cancer, while others have found no significant association. The latest research suggests that even light alcohol consumption, defined as one drink or less per day, may slightly increase the risk of breast cancer. However, the overall risk is still relatively low and it is unclear exactly how much alcohol is considered safe. It is important for individuals to make informed decisions about their alcohol intake based on their individual health and risk factors.'